# Kyber: Un Sistema Criptográfico Post-Cuántico

Kyber es un sistema criptográfico diseñado para proteger las comunicaciones incluso contra ataques de computadoras cuánticas. Se basa en matemáticas avanzadas, específicamente en **retículas**, y permite que dos partes establezcan una clave secreta de forma segura.

## Niveles de Seguridad de Kyber

Kyber ofrece diferentes niveles de seguridad, ajustando un parámetro llamado `k`:

- **Kyber512**: 128 bits de seguridad
- **Kyber768**: 192 bits de seguridad
- **Kyber1024**: 256 bits de seguridad

Lo innovador de Kyber es que, en lugar de cambiar varios parámetros matemáticos, simplemente ajusta el valor de `k` para mejorar la seguridad. Esto hace que sus cálculos sean más eficientes y reutilizables.

---

## 1. (Re)introducción a los Polinomios

Kyber utiliza polinomios en sus cálculos. Un polinomio es una expresión matemática con términos como:

\[ P(X) = 5X^3 + 2X^2 + X + 7 \]

Este polinomio se puede representar como una lista de coeficientes:

\[ P(X) = [7, 1, 2, 5] \]

### Operaciones con Polinomios

Kyber necesita realizar varias operaciones con polinomios, como **suma**, **resta** y **multiplicación**. A continuación, se explican estas operaciones con ejemplos.

---

## 2. Campos y Anillos: Estructuras Matemáticas de Kyber

Para que los cálculos sean eficientes y seguros, Kyber utiliza dos estructuras matemáticas:

1. **Campos Finitos**: Trabajan con números en un rango limitado usando módulo \( q \).
2. **Anillos Polinomiales**: Limitan los polinomios mediante módulo \( X^n + 1 \).

### Ejemplo de un Campo Finito

Supongamos que usamos el número primo \( q = 7 \). Esto significa que cualquier número mayor a 7 se "reduce" con el módulo 7.

- Si tenemos el número 9:
  \[ 9 \mod 7 = 2 \]

- Si tenemos -3:
  \[ -3 \mod 7 = 4 \]

Esto asegura que todos los números estén en el rango de 0 a 6.

### Ejemplo de un Anillo Polinomial

Si definimos el anillo con \( X^4 + 1 = 0 \), significa que cualquier \( X^4 \) en un cálculo se puede reemplazar por \(-1\).

Por ejemplo:

\[ P(X) = X^4 + 3X^2 + 2 \]

Como \( X^4 = -1 \), podemos reescribir:

\[ P(X) = -1 + 3X^2 + 2 \]
\[ P(X) = 3X^2 + 1 \]

Esto reduce el tamaño de los cálculos y mejora la eficiencia de Kyber.

---

## 3. Operaciones con Polinomios

Kyber necesita realizar **suma**, **resta** y **multiplicación** con polinomios dentro del campo finito. A continuación, se muestran ejemplos de cómo funcionan estas operaciones.

### Suma de Polinomios

La suma de polinomios se realiza sumando sus coeficientes, asegurándose de aplicar el módulo \( q \).

**Ejemplo:**

Sumemos los siguientes polinomios en \( \mathbb{Z}_7 \):

\[ P(X) = [3, 2, 1] \]
\[ Q(X) = [4, 1, 0] \]

Hacemos la suma coeficiente por coeficiente:

\[ (3 + 4, 2 + 1, 1 + 0) = [7, 3, 1] \]

Ahora aplicamos el módulo 7:

\[ 7 \mod 7 = 0 \]

Resultado final:

\[ [0, 3, 1] \]

### Resta de Polinomios

La resta se realiza restando coeficientes y aplicando módulo \( q \).

**Ejemplo:**

Restemos los mismos polinomios:

\[ P(X) = [3, 2, 1] \]
\[ Q(X) = [4, 1, 0] \]

\[ (3 - 4, 2 - 1, 1 - 0) = [-1, 1, 1] \]

Aplicamos módulo 7 para eliminar negativos:

\[ -1 \mod 7 = 6 \]

Resultado final:

\[ [6, 1, 1] \]

### Multiplicación de Polinomios

La multiplicación se realiza normalmente y luego se aplica el módulo \( X^n + 1 \).

**Ejemplo:**

Multipliquemos los polinomios en \( \mathbb{Z}_7 \) con \( X^3 + 1 = 0 \):

\[ P(X) = X + 2 \]
\[ Q(X) = X^2 + 3X + 1 \]

Multiplicamos:

\[ (X + 2)(X^2 + 3X + 1) = X^3 + 3X^2 + X + 2X^2 + 6X + 2 \]

Agrupamos términos:

\[ X^3 + 5X^2 + 7X + 2 \]

Ahora aplicamos módulo \( X^3 + 1 = 0 \), lo que significa que podemos sustituir \( X^3 = -1 \):

\[ -1 + 5X^2 + 7X + 2 \]

Como \( 7X \equiv 0X \mod 7 \), eliminamos ese término y simplificamos:

\[ 1 + 5X^2 \]

Resultado final:

\[ 1 + 5X^2 \]

In [15]:
import numpy as np
from numpy.polynomial.polynomial import Polynomial

import random

In [16]:
#cada polinomios es representado como una lista  de valores 

def add_poly(a, b, q):
  #suma polinomios modulo q
  result = [0] * max(len(a), len(b))
  for i in range(max(len(a), len(b))):
    if i < len(a):
      result[i] += a[i]
    if i < len(b):
      result[i] += b[i]
    result[i] %= q
  return result

# inverso aditivo de un polinomio 
def inv_poly(a, q):
  return list(map(lambda x: -x % q, a))

# resta de un polinomio 

def sub_poly(a, b, q):
  return add_poly(a, inv_poly(b, q), q)

# multiplicación modulo q y reducido a f = X**n + 1
def mul_poly_simple(a, b, f, q):
  tmp = [0] * (len(a) * 2 - 1) # the product of two degree n polynomial cannot exceed 2n
  
  # schoolbook multiplication
  for i in range(len(a)):
    # perform a_i * b
    for j in range(len(b)):
      tmp[i + j] += a[i] * b[j]
  
  # take polynomial modulo f
  # since Kyber's poly modulus is always x^n + 1,
  # we can efficiently compute the remainder
  degree_f = len(f) - 1
  for i in range(degree_f, len(tmp)):
    tmp[i - degree_f] -= tmp[i]
    tmp[i] = 0

  # take coefficients modulo q
  tmp = list(map(lambda x: x % q, tmp))
  return tmp[:degree_f]

In [17]:
np.random.seed(0xdeadbeef)

def sign_extend(poly, degree):
  if len(poly) >= degree:
    return poly
  
  return [0] * (degree - len(poly))

def test_mul_poly(N, f, q):
  degree_f = len(f) - 1

  for i in range(N):
    a = (np.random.random(degree_f) * q).astype(int)
    b = (np.random.random(degree_f) * q).astype(int)
    
    a_mul_b = mul_poly_simple(a, b, f, q)
    
    # NumPy reference poly multiplication
    # note that we need to convert the coefficients to int and extend the list to match the fixed size of our impl
    a_mul_b_ref = list(map(lambda x: int(x) % q, ((Polynomial(a) * Polynomial(b)) % Polynomial(f)).coef))
    a_mul_b_ref = sign_extend(a_mul_b_ref, degree_f)

    assert(a_mul_b == a_mul_b_ref)

test_mul_poly(100, [1, 0, 0, 0, 1], 17)


In [18]:
def add_vec(v0, v1, q):
  assert(len(v0) == len(v1)) # sizes need to be the same

  result = []

  for i in range(len(v0)):
    result.append(add_poly(v0[i], v1[i], q))
  
  return result


def mul_vec_simple(v0, v1, f, q):
  assert(len(v0) == len(v1)) # sizes need to be the same

  degree_f = len(f) - 1
  result = [0 for i in range(degree_f - 1)]

  # textbook vector inner product
  for i in range(len(v0)):
    result = add_poly(result, mul_poly_simple(v0[i], v1[i], f, q), q)
  
  return result


def mul_mat_vec_simple(m, a, f, q):
  result = []
  
  # textbook matrix-vector multiplication
  for i in range(len(m)):
    result.append(mul_vec_simple(m[i], a, f, q))
  
  return result


def transpose(m):
  result = [[None for i in range(len(m))] for j in range(len(m[0]))]

  for i in range(len(m)):
    for j in range(len(m[0])):
      result[j][i] = m[i][j]
  
  return result

In [19]:
np.random.seed(0xdeadbeef)

def test_mul_vec(N, k, f, q):
  degree_f = len(f) - 1

  for i in range(N):
    m = (np.random.random([k, k, degree_f]) * q).astype(int)
    a = (np.random.random([k, degree_f]) * q).astype(int)

    m_mul_a = mul_mat_vec_simple(m, a, f, q)

    m_poly = list(map(lambda x: list(map(Polynomial, x)), m))
    a_poly = list(map(Polynomial, a))
    prod = np.dot(m_poly, a_poly)
    m_mul_a_ref = list(map(lambda x: list(map(lambda y: int(y) % q, sign_extend((x % Polynomial(f)).coef, degree_f))), prod))

    assert(m_mul_a == m_mul_a_ref)

test_mul_vec(100, 2, [1, 0, 0, 0, 1], 17)

2. Cifrado de clave pública interno de Kyber
Ahora que hemos definido todos los primitivos subyacentes que necesitamos, ¡podemos avanzar para implementar el primitivo PKE subyacente de Kyber!

# Generación de una matriz \( A \)

Primero, se toma una matriz \( A \) de dimensión \( a \times a \), es decir, una matriz cuadrada de tamaño \( a \). Esta matriz se genera de manera aleatoria y sus elementos son números que pertenecen a un cuerpo finito o un anillo de polinomios.

**Cuerpo finito o anillo de polinomios:** En este tipo de criptografía, se realizan operaciones aritméticas como suma, multiplicación y reducción (es decir, las operaciones se hacen dentro de un conjunto de números "limitado", como los números módulo un número primo o dentro de un anillo de polinomios).

# Generación de los vectores \( s \) y \( m \)

Se generan dos vectores \( s \) y \( m \) (denominados generalmente "secreto" y "mensaje") que también son representaciones de polinomios. Estos vectores tienen un tamaño específico y los coeficientes de los polinomios en estos vectores son pequeños. Este detalle es importante porque garantiza que los coeficientes no sean demasiado grandes, lo cual tiene implicaciones para la eficiencia y seguridad del esquema.

- \( s \): Es el vector secreto. Este vector se utiliza para realizar las operaciones de cifrado y descifrado, y se mantiene secreto.
- \( m \): Es el vector mensaje. En el contexto de la generación de claves, este vector generalmente no se utiliza directamente para transmitir el mensaje, sino como parte de la creación de la clave pública.

# Multiplicación y suma: \( A \cdot s + m \)

Luego, realizamos una multiplicación entre la matriz \( A \) y el vector secreto \( s \), lo que genera un nuevo vector. Esto se hace de manera similar a una multiplicación de matrices en álgebra lineal.

El resultado de \( A \cdot s \) es un vector nuevo. Después, se le suma el vector \( m \), que representa el mensaje. La expresión completa es:

\[
a = A \cdot s + m
\]

Aquí:

- \( a \) es el vector resultante de la clave pública, que es lo que se comparte con los demás para el cifrado.
- \( A \cdot s \) es el resultado de la multiplicación de la matriz \( A \) con el vector secreto \( s \), lo que genera un nuevo vector "secreto".
- \( m \) se suma a este resultado para dar un vector \( a \), que es la clave pública.

# Clave pública y clave secreta

- **Clave pública:** El vector \( a \) es la clave pública. Esta es la que se puede compartir públicamente con cualquier persona que quiera cifrar un mensaje. Cualquiera que tenga acceso a la clave pública puede cifrar mensajes, pero no puede descifrarlos sin la clave secreta.

- **Clave secreta:** El vector \( s \) es la clave secreta. Solo el propietario de esta clave la tiene y la usa para descifrar los mensajes cifrados con la clave pública.

# Dificultad del Problema M-LWE

El problema de Learning With Errors (LWE) y su variante Module Learning With Errors (M-LWE) se basa en la dificultad de resolver sistemas de ecuaciones lineales con errores. Para entender por qué es computacionalmente difícil, examinemos algunos aspectos clave:

## Estructura del Problema

El problema M-LWE se puede formular como:

Dados:

- Una matriz 𝐴 de dimensión 𝑛×𝑚.
- Un vector de salida 𝑎 que se calcula como 𝑎 = 𝐴 ⋅ 𝑠 + m, donde 𝑠 es el vector secreto y 𝑒 es un vector de ruido aleatorio.

El objetivo es recuperar 𝑠 dado 𝐴 y 𝑎.

## Ruido y Enmascaramiento de la Información

La presencia del vector de ruido 𝑒 complica la recuperación de 𝑠:

- **Efecto del Ruido**: Si 𝑒 tiene componentes pequeños (es decir, los elementos de 𝑒 están limitados en magnitud), entonces puede ser visto como un "error" que distorsiona la solución exacta de la ecuación 𝐴 ⋅ 𝑠 = 𝑎 − m. Sin embargo, 𝑒 es suficientemente grande para que la solución no sea trivial de calcular.

## Complejidad Computacional

### a. Algoritmos Clásicos

Para resolver M-LWE, un atacante podría intentar diferentes enfoques:

- **Búsqueda Exhaustiva**: Probar todos los posibles vectores 𝑠 para ver cuál satisface la ecuación. Sin embargo, el número de posibles vectores crece exponencialmente con la dimensión del espacio. Si 𝑠 es de dimensión 𝑛 y cada elemento puede tomar 𝑞 valores (en un campo de tamaño 𝑞), el número total de posibles 𝑠 es 𝑞ⁿ.

  Total de posibles 𝑠 = 𝑞ⁿ
  
- **Reducción a Problemas Conocidos**: Algunos algoritmos intentan reducir el problema M-LWE a otros problemas como el problema de la reducción de redes, pero estos enfoques también son computacionalmente intensivos.

### b. Algoritmos Cuánticos

- **Complejidad de Resolución Cuántica**: Aunque las computadoras cuánticas pueden ejecutar algoritmos como el algoritmo de Grover, que permite buscar en un espacio no estructurado en tiempo 𝑂(𝑁), aún se necesita un número exponencial de recursos para resolver problemas de M-LWE en instancias prácticas.


# Proceso de Encripción en Kyber

## 1. Entradas necesarias
- **Clave pública:** Un par \((A,a)\), donde:
  - \(A\) es una matriz de polinomios.
  - \(a\) es un vector de polinomios.
- **Mensaje:** Un mensaje binario \(m\). Por ejemplo, 
  - \(m = 11 = (1011)_2\).

### Parámetros del sistema:
- \(q\): Tamaño del campo finito (por ejemplo, \(q = 17\)).
- \(\left\lfloor \frac{q}{2} \right\rfloor\): Factor de escalado (por ejemplo, \(9\) para \(q = 17\)).

## 2. Codificación del mensaje
El mensaje binario \(m\) se convierte en un polinomio binario \(m_b\). Por ejemplo:

Si \(m = 11 = (1011)_2\), entonces:

\[ 
m_b = x^3 + x + 1 
\]

(Aquí, los coeficientes del polinomio representan los bits del mensaje).

Luego, este polinomio se escala multiplicándolo por \(\left\lfloor \frac{q}{2} \right\rfloor\):

\[ 
\left\lfloor \frac{q}{2} \right\rfloor m_b = 9 \cdot (x^3 + x + 1) = 9x^3 + 9x + 9 
\]

## 3. Generación de elementos aleatorios
Para el cifrado, se generan los siguientes elementos aleatorios con coeficientes pequeños:

- **Vector cegador** \((a)\): Un vector de polinomios. Por ejemplo:

\[ 
a = [a_1, a_2] = [x + 1, 2x^2] 
\]

- **Vector de error** \((e_1)\): Un vector de polinomios. Por ejemplo:

\[ 
e_1 = [e_{11}, e_{12}] = [x, x^2 + 1] 
\]

- **Polinomio de error** \((e_2)\): Un polinomio. Por ejemplo:

\[ 
e_2 = x^2 + x 
\]

## 4. Cálculo del texto cifrado
El texto cifrado consiste en dos componentes: un vector \(u\) y un polinomio \(v\).

### 4.1. Cálculo de \(u\)

\[ 
u = A^T a + e_1 
\]

- \(A^T\) es la transpuesta de la matriz \(A\).
- \(a\) es el vector cegador.
- \(e_1\) es el vector de error.


In [20]:
def encrypt(A, t, m_b, f, q, r, e_1, e_2):
  half_q = int(q / 2 + 0.5)
  m = list(map(lambda x: x * half_q, m_b))

  u = add_vec(mul_mat_vec_simple(transpose(A), r, f, q), e_1, q)
  v = sub_poly(add_poly(mul_vec_simple(t, r, f, q), e_2, q), m, q)

  return u, v

# Descifrado

Tras recibir el texto cifrado, podemos ver cómo descifrarlo y recuperar el mensaje original.

Kyber define el descifrado PKE interno de la siguiente manera:


​
 
​![Descripción de la imagen](public/dec.png)

 

We

Podemos ver que los términos de ruido restantes \(mi \cdot a, mi_2, s \cdot mi_1\) son todos relativamente pequeños porque se muestrean con “coeficientes pequeños”. Y, por otro lado, \(\left\lfloor \frac{q}{2} \right\rfloor m_b\) es relativamente grande ya que se está escalando para tener una amplitud de la mitad del tamaño del campo \(q\).

Por lo tanto, para recuperar \(m_b\) de \(mensaje\), simplemente realizamos una operación de “redondeo” y vemos si cada coeficiente en \(mensaje\) está más cerca de \(\left\lfloor \frac{q}{2} \right\rfloor m_b\) o 0. Finalmente, los resultados de la comparación generan un vector booleano que debería coincidir con el inicial \(m_b\).

De manera similar, implementamos el descifrado mediante el cálculo \(mensaje\) y realizar manualmente la operación circular.


In [21]:
def decrypt(s, u, v, f, q):
  m_n = sub_poly(v, mul_vec_simple(s, u, f, q), q)

  half_q = int(q / 2 + 0.5)
  def round(val, center, bound):
    dist_center = np.abs(center - val)
    dist_bound = min(val, bound - val)
    return center if dist_center < dist_bound else 0

  m_n = list(map(lambda x: round(x, half_q, q), m_n))
  m_b = list(map(lambda x: x // half_q, m_n))
  
  return m_b

Test


In [22]:
q = 17 # plain modulus
f = [1, 0, 0, 0, 1] # poly modulus, x**4 + 1

s = [[0, 1, -1, -1], [0, -1, 0, -1]] # secret key, [-x**3-x**2+x, -x**3-x]
A = [[[11, 16, 16, 6], [3, 6, 4, 9]], [[1, 10, 3, 5], [15, 9, 1, 6]]] # public key
e = [[0, 0, 1, 0], [0, -1, 1, 0]] # noise
m_b = [1, 1, 0, 1] # message in binary

t = add_vec(mul_mat_vec_simple(A, s, f, q), e, q)

r = [[0, 0, 1, -1], [-1, 0, 1, 1]] # blinding vector for encrypt
e_1 = [[0, 1, 1, 0], [0, 0, 1, 0]] # noise vector for encrypt
e_2 = [0, 0, -1, -1] # noise poly for encrypt

u, v = encrypt(A, t, m_b, f, q, r, e_1, e_2)
m_b2 = decrypt(s, u, v, f, q)

assert(m_b == m_b2)

In [23]:
np.random.seed(0xdeadbeef)

def test_enc_dec(N, k, f, q):
  degree_f = len(f) - 1

  A = (np.random.random([k, k, degree_f]) * q).astype(int)
  s = (np.random.random([k, degree_f]) * 3).astype(int) - 1
  e = (np.random.random([k, degree_f]) * 3).astype(int) - 1
  t = add_vec(mul_mat_vec_simple(A, s, f, q), e, q)

  failed = 0

  for i in range(N):
    m_b = (np.random.random(degree_f) * 2).astype(int)

    r = (np.random.random([k, degree_f]) * 3).astype(int) - 1
    e_1 = (np.random.random([k, degree_f]) * 3).astype(int) - 1
    e_2 = (np.random.random([degree_f]) * 3).astype(int) - 1

    u, v = encrypt(A, t, m_b, f, q, r, e_1, e_2)
    m_b2 = decrypt(s, u, v, f, q)

    if m_b.tolist() != m_b2:
      failed += 1
  
  print(f"[k={k}, f={f}, q={q}] Test result: {failed}/{N} failed decryption!")

test_enc_dec(100, 2, [1, 0, 0, 0, 1], 17)
test_enc_dec(100, 2, [1, 0, 0, 0, 1], 37)
test_enc_dec(100, 2, [1, 0, 0, 0, 1], 67)

[k=2, f=[1, 0, 0, 0, 1], q=17] Test result: 27/100 failed decryption!
[k=2, f=[1, 0, 0, 0, 1], q=37] Test result: 1/100 failed decryption!
[k=2, f=[1, 0, 0, 0, 1], q=67] Test result: 0/100 failed decryption!


In [24]:
import numpy as np
from numpy.polynomial.polynomial import Polynomial

# Función para generar una matriz de polinomios aleatorios
def generate_random_poly_matrix(k, degree_f, q):
    return (np.random.random([k, k, degree_f]) * q).astype(int)

# Función para generar un vector de polinomios aleatorios
def generate_random_poly_vector(k, degree_f, q):
    return (np.random.random([k, degree_f]) * q).astype(int)

# Parámetros de Kyber
k = 2  # Dimensión de la matriz A
degree_f = 4  # Grado del polinomio f(X) = X^4 + 1
q = 17  # Campo finito GF(17)

# Generar la matriz A y los vectores s, e
A = generate_random_poly_matrix(k, degree_f, q)
s = generate_random_poly_vector(k, degree_f, q)
e = generate_random_poly_vector(k, degree_f, q)

# Calcular t = A * s + e
t = add_vec(mul_mat_vec_simple(A, s, [1, 0, 0, 0, 1], q), e, q)

# Clave pública: (A, t)
public_key = (A, t)
# Clave privada: s
private_key = s

In [25]:
# Mensaje a cifrar (en forma de polinomio binario)
m_b = [1, 0, 1, 1]  # Representa el mensaje 11 en binario (1011)

# Generar los vectores r, e1, e2 para el cifrado
r = generate_random_poly_vector(k, degree_f, q)
e1 = generate_random_poly_vector(k, degree_f, q)
e2 = (np.random.random(degree_f) * q).astype(int)

# Cifrar el mensaje
u, v = encrypt(A, t, m_b, [1, 0, 0, 0, 1], q, r, e1, e2)

# Mostrar el mensaje cifrado
print("Mensaje cifrado (u):", u)
print("Mensaje cifrado (v):", v)

Mensaje cifrado (u): [[np.int64(12), np.int64(9), np.int64(12), np.int64(4)], [np.int64(11), np.int64(9), np.int64(10), np.int64(10)]]
Mensaje cifrado (v): [np.int64(7), np.int64(6), np.int64(2), np.int64(2)]


In [26]:
# Descifrar el mensaje
decrypted_m_b = decrypt(private_key, u, v, [1, 0, 0, 0, 1], q)

# Mostrar el mensaje descifrado
print("Mensaje descifrado:", decrypted_m_b)

Mensaje descifrado: [0, 0, 0, 0]


In [27]:
import numpy as np
from numpy.polynomial.polynomial import Polynomial

# Funciones de operaciones con polinomios (definidas previamente)
def add_poly(a, b, q):
    result = [0] * max(len(a), len(b))
    for i in range(max(len(a), len(b))):
        if i < len(a):
            result[i] += a[i]
        if i < len(b):
            result[i] += b[i]
        result[i] %= q
    return result

def inv_poly(a, q):
    return list(map(lambda x: -x % q, a))

def sub_poly(a, b, q):
    return add_poly(a, inv_poly(b, q), q)

def mul_poly_simple(a, b, f, q):
    tmp = [0] * (len(a) * 2 - 1)
    for i in range(len(a)):
        for j in range(len(b)):
            tmp[i + j] += a[i] * b[j]
    degree_f = len(f) - 1
    for i in range(degree_f, len(tmp)):
        tmp[i - degree_f] -= tmp[i]
        tmp[i] = 0
    tmp = list(map(lambda x: x % q, tmp))
    return tmp[:degree_f]

def add_vec(v0, v1, q):
    assert(len(v0) == len(v1))
    result = []
    for i in range(len(v0)):
        result.append(add_poly(v0[i], v1[i], q))
    return result

def mul_vec_simple(v0, v1, f, q):
    assert(len(v0) == len(v1))
    degree_f = len(f) - 1
    result = [0 for i in range(degree_f - 1)]
    for i in range(len(v0)):
        result = add_poly(result, mul_poly_simple(v0[i], v1[i], f, q), q)
    return result

def mul_mat_vec_simple(m, a, f, q):
    result = []
    for i in range(len(m)):
        result.append(mul_vec_simple(m[i], a, f, q))
    return result

def transpose(m):
    result = [[None for i in range(len(m))] for j in range(len(m[0]))]
    for i in range(len(m)):
        for j in range(len(m[0])):
            result[j][i] = m[i][j]
    return result

def encrypt(A, t, m_b, f, q, r, e_1, e_2):
    half_q = int(q / 2 + 0.5)
    m = list(map(lambda x: x * half_q, m_b))
    u = add_vec(mul_mat_vec_simple(transpose(A), r, f, q), e_1, q)
    v = sub_poly(add_poly(mul_vec_simple(t, r, f, q), e_2, q), m, q)
    return u, v

def decrypt(s, u, v, f, q):
    m_n = sub_poly(v, mul_vec_simple(s, u, f, q), q)
    half_q = int(q / 2 + 0.5)
    def round(val, center, bound):
        dist_center = np.abs(center - val)
        dist_bound = min(val, bound - val)
        return center if dist_center < dist_bound else 0
    m_n = list(map(lambda x: round(x, half_q, q), m_n))
    m_b = list(map(lambda x: x // half_q, m_n))
    return m_b

# Parámetros de Kyber
k = 2
degree_f = 4
q = 17

# Generar claves
A = (np.random.random([k, k, degree_f]) * q).astype(int)
s = (np.random.random([k, degree_f]) * q).astype(int)
e = (np.random.random([k, degree_f]) * q).astype(int)
t = add_vec(mul_mat_vec_simple(A, s, [1, 0, 0, 0, 1], q), e, q)
public_key = (A, t)
private_key = s

# Mensaje a cifrar
m_b = [1, 0, 1, 1]  # 11 en binario (1011)

# Generar r, e1, e2 para el cifrado
r = (np.random.random([k, degree_f]) * q).astype(int)
e1 = (np.random.random([k, degree_f]) * q).astype(int)
e2 = (np.random.random(degree_f) * q).astype(int)

# Cifrar el mensaje
u, v = encrypt(A, t, m_b, [1, 0, 0, 0, 1], q, r, e1, e2)
print("Mensaje cifrado (u):", u)
print("Mensaje cifrado (v):", v)

# Descifrar el mensaje
decrypted_m_b = decrypt(private_key, u, v, [1, 0, 0, 0, 1], q)
print("Mensaje descifrado:", decrypted_m_b)

Mensaje cifrado (u): [[np.int64(16), np.int64(8), np.int64(1), np.int64(16)], [np.int64(11), np.int64(16), np.int64(5), np.int64(0)]]
Mensaje cifrado (v): [np.int64(8), np.int64(16), np.int64(16), np.int64(15)]
Mensaje descifrado: [0, 0, 0, 0]
